In [1]:
import pandas as pd
import numpy as np

Download link: 
[DP02](https://data.census.gov/cedsci/table?g=0500000US36005.140000,36047.140000,36061.140000,36081.140000,36085.140000&tid=ACSDP5Y2018.DP02&moe=false&hidePreview=true),
[DP04](https://data.census.gov/cedsci/table?g=0500000US36005.140000,36047.140000,36061.140000,36081.140000,36085.140000&tid=ACSDP5Y2018.DP04&moe=false&hidePreview=true),
[DP03](https://data.census.gov/cedsci/table?q=unemployment&g=0500000US36005.140000,36047.140000,36061.140000,36081.140000,36085.140000&tid=ACSDP5Y2018.DP03&hidePreview=true),
[DP05](https://data.census.gov/cedsci/table?q=race&g=0400000US36_0500000US36005.140000,36047.140000,36061.140000,36081.140000,36085.140000&tid=ACSDP5Y2018.DP05&moe=false&hidePreview=true)

In [122]:
path = '../data/neighborhood_char/'
acs_edu = pd.read_csv(path+'ACSDP5Y2018.DP02_data_with_overlays.csv', low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','DP02_0066PE','DP02_0067PE']).\
                            rename(columns={'DP02_0066PE':'non_high_school_per','DP02_0067PE':'college_per'})
acs_edu[['non_high_school_per','college_per']] = acs_edu[['non_high_school_per','college_per']].\
                                                replace(r'[-+,*]',np.nan,regex=True).astype(float)
acs_edu['non_high_school_per'] = 100 - acs_edu['non_high_school_per']
# % High school graduate or higher, % Bachelor's degree or higher

acs_income = pd.read_csv(path+'ACSDP5Y2018.DP03_data_with_overlays.csv', low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','DP03_0062E']).\
                            rename(columns={'DP03_0062E':'med_income'})
acs_income[['med_income']] = acs_income[['med_income']].replace(r'[-+,*]',np.nan,regex=True).astype(float)
# Median household income

acs_rent = pd.read_csv(path+'ACSDP5Y2018.DP04_data_with_overlays.csv', low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','DP04_0047E', 'DP04_0047PE', 'DP04_0134E','DP04_0142PE']).\
                            rename(columns={'DP04_0047E':'renter_num','DP04_0047PE':'renter_per',
                                            'DP04_0134E':'mid_rent','DP04_0142PE':'rent_35'})
acs_rent[['renter_num','renter_per','mid_rent','rent_35']] = acs_rent[['renter_num','renter_per','mid_rent','rent_35']].\
                                                            replace(r'[-+,*]',np.nan,regex=True).astype(float)
# Renter-occupied, % Renter-occupied, Median rent, % rent cost more than 35.0 percent of income

acs_race = pd.read_csv(path+'ACSDP5Y2018.DP05_data_with_overlays.csv', low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','DP05_0070E','DP05_0071PE','DP05_0077PE','DP05_0078PM','DP05_0080PE']).\
                            rename(columns={'DP05_0070E':'total',
                                            'DP05_0071PE':'latinx',
                                            'DP05_0077PE':'white',
                                            'DP05_0078PM':'black',
                                            'DP05_0080PE':'asian'})
acs_race[['total','latinx','white','black','asian']] = acs_race[['total','latinx','white',
                                                                 'black','asian']].replace(r'[-+,*]',np.nan,
                                                                                           regex=True).astype(float)
# Total population, % Latinx, % White, % Black or African American, % Asian

In [123]:
acs = acs_edu.merge(acs_income).merge(acs_rent).merge(acs_race)
acs['include'] = (acs.renter_num>500)*1
acs.head()

,GEO_ID,non_high_school_per,college_per,med_income,renter_num,renter_per,mid_rent,rent_35,total,latinx,white,black,asian,include
0,1400000US36081000100,1.8,79.5,142348.0,3642.0,87.7,3164.0,27.3,8830.0,8.2,58.8,3.9,22.8,1
1,1400000US36081000200,21.9,23.2,72813.0,415.0,44.4,1707.0,40.1,3208.0,70.0,11.7,1.2,16.1,0
2,1400000US36081000400,33.7,22.3,59423.0,633.0,53.3,1554.0,57.9,4583.0,67.0,4.4,2.8,23.2,1
3,1400000US36081000600,26.1,23.8,77407.0,388.0,40.2,1678.0,60.9,3690.0,64.6,10.4,3.3,20.5,0
4,1400000US36081000700,4.0,75.3,126935.0,1783.0,62.5,2223.0,31.3,6079.0,16.7,52.3,0.6,25.2,1


In [124]:
acs_summary = acs.groupby('include').mean().drop('total',axis=1).round(2)
acs_summary['tract_count'] = acs.groupby('include')['GEO_ID'].count()
acs_summary = acs_summary.T.rename(columns={0:'Exclueded',1:'Included'}).rename_axis(None, axis = 1)
acs_summary

,Exclueded,Included
non_high_school_per,16.25,19.82
college_per,32.77,36.40
med_income,77794.30,61512.92
renter_num,258.28,1384.68
renter_per,38.66,75.24
mid_rent,1557.34,1466.73
rent_35,46.99,45.29
latinx,19.69,31.03
white,34.85,31.05
black,5.74,4.62


In [125]:
acs.to_csv(path+'acs_summary.csv', index=False)